In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets==1.0.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import json
import torch
import os
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'):
#   os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

In [5]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [6]:
from datasets import load_dataset

# Download the SQuAD dataset
dataset = load_dataset('squad')
print(dataset)
print()
# Print the first example in the training set
print(dataset['train'][0])
print()
# Access the context and question of the first example
context = dataset['train'][0]['context']
question = dataset['train'][0]['question']

# Print the context and question
print('Context:', context)
print()
print('Question:', question)


Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


DatasetDict({'train': Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599), 'validation': Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 10570)})

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is 

In [7]:
train_data = dataset['train']
train_data

Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599)

In [8]:
len(train_data)

87599

In [9]:
print(train_data[0])

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '5733be284776f41900661182', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'title': 'University_of_Notre_Dame'}


In [10]:
print(train_data[0]['context'])
print(train_data[0]['question'])
print(train_data[0]['answers']['text'][0])

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Saint Bernadette Soubirous


In [11]:
print(train_data[99]['context'])


One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record of 105 wins, 12 losses, and five ties. During his 13 years the Irish won three national championships, had five undefeated seasons, won the Rose Bowl in 1925, and produced players such as George Gipp and the "Four Horsemen". Knute Rockne has the highest winning percentage (.881) in NCAA Division I/FBS football history. Rockne's offenses employed the Notre Dame Box and his defenses ran a 7–2–2 scheme. The last game Rockne coached was on December 14, 1930 when he led a group of Notre Dame all-stars against the New York Giants in New York City.


In [12]:
%%time
context = []
question = []
answer = []

for i in range(len(train_data)):

    context.append(train_data[i]['context'])
    question.append(train_data[i]['question'])
    answer.append(train_data[i]['answers'])

CPU times: user 17.1 s, sys: 216 ms, total: 17.3 s
Wall time: 19.4 s


In [13]:
print(len(context))
print(len(question))
print(len(answer))

87599
87599
87599


In [14]:
train_size = 4000
test_size = 500
train_contexts, train_questions, train_answers = context[0:train_size], question[0:train_size], answer[0:train_size]
valid_contexts, valid_questions, valid_answers = context[train_size:train_size+test_size], question[train_size:train_size+test_size], answer[train_size:train_size+test_size]

In [15]:
print(len(train_contexts))
print(len(train_questions))
print(len(train_answers))


4000
4000
4000


In [16]:
print(len(valid_contexts))
print(len(valid_questions))
print(len(valid_answers))

500
500
500


In [17]:
# print a random question and answer
print(f'There are {len(train_questions)} questions')
print(train_questions[-1000])
print(train_answers[-1000])

There are 4000 questions
What place did the game take in Nintendo's Official list of 100 Greatest Nintendo Games of All Time?
{'answer_start': [660], 'text': ['16th']}


In [18]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0] #660
    answer['answer_start'] = start_idx
    end_idx = start_idx + len(gold_text)
    # print(start_idx,gold_text,end_idx)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)

In [19]:
valid_answers[0]

{'answer_start': 727, 'text': ['July 19, 1909'], 'answer_end': 740}

In [20]:
# You can see that now we get the answer_end also
print(train_questions[-1000])
print(train_answers[-1000])

What place did the game take in Nintendo's Official list of 100 Greatest Nintendo Games of All Time?
{'answer_start': 660, 'text': ['16th'], 'answer_end': 664}


In [21]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True, max_length = 512)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True, max_length = 512)

In [22]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [23]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 4000 context-question pairs


In [24]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [ ]:
# Answer ==>Charater starrting and ending points ==> 100,120
# Start_Position ==> 25
# End_postion ==>  29

#  Data we have is==>  answers in character tokens 
# Question, Context ===>  Answers(words)

In [ ]:
'''
encoding has start and end of character indices
after the code
encoding has start and end of the word indices

'''
# answer_start ==> 100  character indices
#answer_end ==> 120   indices
# encodings.char_to_token(i, answers[i]['answer_end'] - 1  ===> fetch the word index

In [25]:
def add_token_positions(encodings, answers):
  start_positions = []  # Here, we ar talking word starting position
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'])) #9
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))  #13
  # end_positions.append(13) #13 word index
  

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)

In [26]:
train_encodings['start_positions'][:10]

[114, 40, 65, 85, 20, 51, 85, 111, 27, 166]

In [27]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [28]:
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

### **Dataloaders 🔁**

In [29]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

## Fine Tune

In [30]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [31]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [32]:
%%time
from transformers import AdamW

N_EPOCHS = 2
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 2: 100%|██████████| 250/250 [06:43<00:00,  1.61s/it, loss=1.04]

CPU times: user 13min 12s, sys: 3.19 s, total: 13min 16s
Wall time: 13min 30s


In [ ]:
# # 1,2,3,4,5....,10
# Weights = 0
# 0.1
# 0.12
# 0.20
# ...
# 0.45

# Epoch 2

# 0.47
# 0.49

In [33]:
loss

tensor(1.0422, device='cuda:0', grad_fn=<DivBackward0>)

**Save the model in my drive in order not to run it each time**

In [34]:
#model_path = '/content/drive/MyDrive/BERT-SQuAD'
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)

**Respectively, load the saved model**

In [35]:
#from transformers import BertForQuestionAnswering, BertTokenizerFast

#model_path = '/content/drive/MyDrive/BERT-SQuAD'
#model = BertForQuestionAnswering.from_pretrained(model_path)
#tokenizer = BertTokenizerFast.from_pretrained(model_path)

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(f'Working on {device}')

#model = model.to(device)

### **Test**

In [36]:
# model.eval()

# acc = []
# predict_start_index = []
# predict_end_index = []

# for batch in tqdm(valid_loader):
#   with torch.no_grad():
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     start_true = batch['start_positions'].to(device)
#     end_true = batch['end_positions'].to(device)
    
#     outputs = model(input_ids, attention_mask=attention_mask)

#     start_pred = torch.argmax(outputs['start_logits'], dim=1)
#     end_pred = torch.argmax(outputs['end_logits'], dim=1)

#     predict_start_index.append(start_pred)
#     predict_end_index.append(end_pred)


100%|██████████| 32/32 [00:17<00:00,  1.81it/s]


In [37]:
def get_prediction(question,context):
    model.eval()

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    inputs.to(device)

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(outputs[1]) + 1 

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer

In [38]:
type(valid_contexts)

list

In [39]:
len(valid_contexts)

500

In [40]:
valid_contexts[50]

"Manhattan's skyline, with its many skyscrapers, is universally recognized, and the city has been home to several of the tallest buildings in the world. As of 2011, New York City had 5,937 high-rise buildings, of which 550 completed structures were at least 330 feet (100 m) high, both second in the world after Hong Kong, with over 50 completed skyscrapers taller than 656 feet (200 m). These include the Woolworth Building (1913), an early gothic revival skyscraper built with massively scaled gothic detailing."

In [41]:
valid_questions[50]

'In what year was the Woolworth Building completed?'

In [42]:
valid_answers[50]

{'answer_start': 425, 'text': ['1913'], 'answer_end': 429}

In [43]:
answer = get_prediction(valid_questions[50], valid_contexts[50])
answer

'1913'

In [44]:
original_answer = []

for i in range(len(valid_answers)):

     original_answers = valid_answers[i]['text'][0]
     original_answer.append(original_answers)

In [45]:
predicted_answer = []

for i in range(len(valid_questions)):

     answer = get_prediction(valid_questions[i], valid_contexts[i])
     predicted_answer.append(answer)

In [46]:
import pandas as pd
# Create a dictionary from the lists
data = {'Question': valid_questions, 'Context': valid_contexts, 'Original_Answer': original_answer, 'Predicted_Answer': predicted_answer}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

In [47]:
df.sample(10)

,Question,Context,Original_Answer,Predicted_Answer
212,What borough has a Gini Coefficient of 0.6?,New York City has a high degree of income disp...,Manhattan,new york city has a high degree of income disp...
235,NYC's fashion industry employs how many people?,"As of 2013, the global advertising agencies of...","180,000","180, 000"
363,What is the nickname given to New York City Po...,The New York City Police Department (NYPD) has...,New York's Finest,finest
325,When was the Manhattan Neighborhood Network cr...,New York is also a major center for non-commer...,1971,"wnyc, a public radio station owned by the city..."
477,"In minutes, how long does it take for the aver...",Public transport is essential in New York City...,38.4,
92,The highest temperature ever recorded in NYC w...,"Winters are cold and damp, and prevailing wind...",1936,cold and damp
170,From what country did the largest number of fo...,Approximately 37% of the city's population is ...,Dominican Republic,37 % of the city ' s population is foreign bor...
422,In what year were the Brooklyn Dodgers founded?,New York City is home to the headquarters of t...,1882,1882
192,"Out of all African nations, which provided the...","Ecuador, Colombia, Guyana, Peru, and Brazil we...",Egypt,
213,What previous mayor of New York is a billionaire?,New York City has a high degree of income disp...,Michael R. Bloomberg,4. 6 %


In [48]:
df.shape

(500, 4)